<h1>Neural Network</h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

Hyperparameters and other variable

In [2]:
# defines the data type of the variable used for w
dtype = torch.float
dtypeVolume = torch.int

# defines the device in which we would like to keep the tensor
device = torch.device("cpu")

#definition of the Neural Network HyperParameters
n_input = None
n_hidden_layer = 5
n_hidden = [10,20,5,15,14]
n_out = 10
learning_rate = 0.1

# Used to enable training analysis through TensorBoard
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []


In [10]:
import ipynb
import datetime
#Importing the function getAllCurrencyMarketElement from the file DatabaseScript.ipynb
from ipynb.fs.full.DatabaseScript import getAllCurrencyMarketElement
#In entry we want a matrix with Open , High , Low , Close and Volume
#In exit we want a array with Close 
t = getAllCurrencyMarketElement("BTC-USD")

date = t['Date']

index = []

for d in date:
        index.append(datetime.datetime.strptime(d[2:10],"%y-%m-%d").date())
    

X = t.drop(columns='Date') #Entry in nn
y = t['Close'] #Output of nn

In [11]:
from sklearn.model_selection import TimeSeriesSplit , train_test_split

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=10,shuffle=False)

print(f"\t--- X shape ---\n[TRAIN] : {X_train.shape} , [TEST] : {X_test.shape}")
print(f"\n\t--- y shape ---\n[TRAIN] : {y_train.shape} , [TEST] : {y_test.shape}\n")

taille_pred = 10
nspl = X_train.shape[0]//taille_pred
tscv = TimeSeriesSplit(n_splits=nspl, test_size=10)
print(tscv)

all_splits = list(tscv.split(X_train,y_train))


# for i in range(len(all_splits)):
#     print(f'Split {i} shape: {len(all_splits[i][0])}')
# X_train.iloc[train_1]
# X_train.iloc[valid_1]

	--- X shape ---
[TRAIN] : (356, 5) , [TEST] : (10, 5)

	--- y shape ---
[TRAIN] : (356,) , [TEST] : (10,)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=35, test_size=10)


Definine NN

In [12]:
def Create_NN(n_input):
    modules = []
    modules.append(nn.Linear(in_features=n_input , out_features=n_hidden[0]))
    modules.append(nn.ReLU())
    for i in range(n_hidden_layer-1):
                modules.append(nn.Linear(in_features=n_hidden[i] , out_features=n_hidden[i+1]))
                modules.append(nn.ReLU())

    modules.append(nn.Linear(in_features=n_hidden[-1], out_features=n_out))


    model = nn.Sequential(*modules)

    return model

In [ ]:
n_epochs = 1000

# training loop
for split in all_splits:
    i_train , i_valid = split
    X_train_valid , y_valid = torch.tensor(X_train['Close'].iloc[i_train].values,dtype=torch.float) , torch.tensor(y_train.iloc[i_valid].values,dtype=torch.float)
    

    model = Create_NN(len(i_train))
    loss_function = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(n_epochs):
        y_pred = model(X_train_valid) 
        loss = loss_function(y_pred, y_valid)
        if epoch % 100 == 0:
            #print(f"Epoch [{epoch+1:4}/{n_epochs}], loss: {loss:.6f}")
            # Write epoch loss for TensorBoard
            writer.add_scalar("Loss/train", loss.item(), epoch)
        
        #Maj best model
        if loss.item() < best_mse:
            best_mse = loss.item()
            best_weights = model.parameters()

        
        model.zero_grad()
        loss.backward()

        with torch.no_grad():
            for param in model.parameters():
                param -= learning_rate * param.grad
    
    print(f"Split training finished. Final loss: {loss:.6f} . Lowest loss : {best_mse:.6f} .")


X_train , y_test = torch.tensor(X_train['Close'].values,dtype=torch.float) , torch.tensor(y_test.values,dtype=torch.float)
y_pred = model(X_train)
loss = loss_function(y_pred,y_test)

print(f"Training finished. Final loss: {loss:.6f} . Lowest loss during training: {best_mse} .")
    

Split training finished. Final loss: nan . Lowest loss : 2159185408.000000 .
Split training finished. Final loss: nan . Lowest loss : 1731731840.000000 .
Split training finished. Final loss: nan . Lowest loss : 1628892416.000000 .
Split training finished. Final loss: nan . Lowest loss : 1480214272.000000 .
Split training finished. Final loss: nan . Lowest loss : 995598976.000000 .
Split training finished. Final loss: nan . Lowest loss : 883053376.000000 .
Split training finished. Final loss: nan . Lowest loss : 883053376.000000 .
Split training finished. Final loss: nan . Lowest loss : 802832768.000000 .
Split training finished. Final loss: nan . Lowest loss : 425755328.000000 .
Split training finished. Final loss: nan . Lowest loss : 406370688.000000 .
Split training finished. Final loss: nan . Lowest loss : 406370688.000000 .
Split training finished. Final loss: nan . Lowest loss : 406370688.000000 .
Split training finished. Final loss: 1283641976029184.000000 . Lowest loss : 4063706

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x356 and 346x10)

In [ ]:
print(y_test.values)

[20632.41015625 22163.94921875 24197.53320312 24746.07421875
 24375.9609375  25052.7890625  27423.9296875  26965.87890625
 28038.67578125 28323.87304688]


In [ ]:
a = np.linspace(1,n_epochs,n_epochs//100).astype(int)

fig , axs = plt.subplots(2,2 , figsize=(16,10))
fig.suptitle("Training Information and Results")

ax1 = axs[0,0]
ax1.plot(a,history)
ax1.set(xlabel="Epoch" , ylabel="Loss",title='Loss Function')

final_result = y_train - y_pred
ax2 = axs[0,1]
ax2.plot(index,final_result.detach().numpy())
ax2.set(ylabel='Difference',xlabel='x')
ax2.set_title('Différence between original and predicted values')

ax3 = axs[1,1]
ax3.plot(index,y_train,x,y_pred.detach().numpy(),'r')
ax3.set(title='Final trained result',ylabel='y_pred',xlabel='x')

ax4 = axs[1,0]
ax4.plot(index,y_train[0:700],'b',x,y_train[700:],'r')
ax4.set(title='Initial values',ylabel='y_train',xlabel='x')